In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
from numpy import log10
from collections import defaultdict
from collections import Counter

In [ ]:
# please fill this fields with your data
# now the fields are filled with default human vs rat example

# please look at example files content
data_dir = os.path.join(os.getcwd(), "data")

# reference genome annotation
ref_annot = os.path.join(data_dir, "ens_annot_example.bed")

# ehain features
chain_feats = os.path.join(data_dir, "features_example.txt")

# ensembl data for reference and query orthologs
ens_data = os.path.join(data_dir, "ensembl_rat_data_example.txt")

# save train to...
train_path = os.path.join(data_dir, "train.tsv")

In [ ]:
# read chain featutes file
df = pd.read_csv(chain_feats, header=0, sep="\t")
print(len(df))

In [ ]:
# get longest isoform of each gene, ignore orther
gene_to_isoforms = defaultdict(list)
isofotm_to_len = {}
longest_isoforms = []
gene_to_longest = {}
longest_to_gene = {}

f = open(ens_data, "r")
for line in f:
    line_data = line.split("\t")
    gene = line_data[0]
    trans = line_data[1].rstrip()
    gene_to_isoforms[gene].append(trans)
f.close()

f = open(ref_annot, "r")
for line in f:
    line_data = line[:-1].split("\t")
    trans = line_data[3]
    block_sizes = line_data[10].split(",")
    overall_CDS = sum(int(x) for x in block_sizes if x != "")
    isofotm_to_len[trans] = overall_CDS
f.close()

for gene, isoforms in gene_to_isoforms.items():
    isof_lens = [(i, isofotm_to_len.get(i)) for i in isoforms if isofotm_to_len.get(i)]
    if len(isof_lens) == 0:
        continue  # no isoforms with length?
    isof_len_sort = sorted(isof_lens, key=lambda x: x[1], reverse=True)
    longest = isof_len_sort[0][0]
    longest_isoforms.append(longest)
    gene_to_longest[gene] = longest
    longest_to_gene[longest] = gene

longest_isoforms = set(longest_isoforms)  # set to search faster

In [ ]:
# filter dataset
# delete non-longest isoforms
df = df[df["gene"].isin(longest_isoforms)]
print(len(df))

In [ ]:
# get list of genes classified by Ensembl as one2one
# left only intersection with longest isoforms
Ens_df = pd.read_csv(ens_data, header=0, sep="\t")
Ens_o2o_df = Ens_df[Ens_df["Rat homology type"] == "ortholog_one2one"]
Ens_one2ones_trans = set(Ens_o2o_df["Transcript stable ID"])
Ens_o2o_longest = longest_isoforms.intersection(Ens_one2ones_trans)
print(len(Ens_o2o_longest))

In [ ]:
# starting positive dataset, left only one-2-ones according the ensembl
pos_1 = df[df["gene"].isin(Ens_o2o_longest)]
# remove small chains / chains that 100% are not orthologs or paralogs
# and genes that covered by > 10 chains also
pos_1 = pos_1[pos_1["gene_overs"] <= 10]
pos_1["ex_covered"] = pos_1["exon_cover"] / pos_1["ex_fract"] * 100
pos_1 = pos_1[pos_1["ex_covered"] > 10]

In [ ]:
# left genes that are covered with only one chain -> 99% rock-solid orthologs
gene_times = Counter(pos_1["gene"])
genes_once = set(k for k, v in gene_times.items() if v == 1)
genes_mult = set(k for k, v in gene_times.items() if v > 1)
pos_2 = pos_1[pos_1["gene"].isin(genes_once)]
pos_2.drop(["ex_covered"], axis=1)
print(len(pos_2))

In [ ]:
# need genes with multiple chains of reasonable length
# but still one-2-one according the ensembl
neg_1 = pos_1[pos_1["gene"].isin(genes_mult)]
# get chains, remove the top chain
gene_to_chains = defaultdict(list)
for elem in zip(neg_1["gene"], neg_1["chain"]):
    gene_to_chains[elem[0]].append(elem[1])

orth_chains = []
for gene, chains in gene_to_chains.items():
    chains_s = sorted(chains)
    orth_chains.append(chains_s[0])

orth_chains = set(orth_chains)
neg_2 = neg_1[~neg_1["chain"].isin(orth_chains)]
print(len(neg_2))

In [ ]:
# add class labels + merge df and save it
pos_2.insert(len(pos_2.columns), "y", 1)
neg_2.insert(len(neg_2.columns), "y", 0)

merged = pd.concat([pos_2, neg_2])
print(len(merged))

merged.to_csv(train_path, sep="\t", index=False)
print(merged.columns)
print(merged[1000:1010])

In [ ]:
# maybe build a couple of plots

plt.rcParams["figure.figsize"] = [12, 6]
feat = "gl_exo"  # green for positive, red for negative sets
merged[merged["y"] == 1][feat].hist(bins=100, alpha=0.5, color="green")
merged[merged["y"] == 0][feat].hist(bins=100, alpha=0.5, color="red")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [12, 6]
merged["chain_len_log"] = log10(merged["chain_len"])
feat = "chain_len_log"  # green for positive, red for negative sets
merged[merged["y"] == 1][feat].hist(bins=100, alpha=0.5, color="green")
merged[merged["y"] == 0][feat].hist(bins=100, alpha=0.5, color="red")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [12, 6]
merged["synt_log"] = log10(merged["synt"])
feat = "synt_log"  # green for positive, red for negative sets
merged[merged["y"] == 1][feat].hist(bins=100, alpha=0.5, color="green")
merged[merged["y"] == 0][feat].hist(bins=100, alpha=0.5, color="red")
plt.show()